In [1]:
#8095996049:AAErQafRQJTwala32AXFrsrbusayifVRJkg

In [2]:
import telebot
from telebot import types
import joblib
import pandas as pd

In [3]:

# Загрузка модели
try:
    model = joblib.load('comment_classifier.joblib')
    class_columns = ['Нравится скорость отработки заявок', 
                   'Нравится качество выполнения заявки', 
                   'Нравится качество работы сотрудников', 
                   'Понравилось выполнение заявки', 
                   'Вопрос решен']
    print("Модель успешно загружена")
except Exception as e:
    print(f"Ошибка загрузки модели: {e}")
    raise

# Инициализация бота
TOKEN = '8095996049:AAErQafRQJTwala32AXFrsrbusayifVRJkg'
bot = telebot.TeleBot(TOKEN)

# Создаем клавиатуру для главного меню
def create_main_menu():
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)
    btn1 = types.KeyboardButton('🔍 Анализ комментария')
    btn2 = types.KeyboardButton('ℹ️ О боте')
    btn3 = types.KeyboardButton('📊 Примеры комментариев')
    btn4 = types.KeyboardButton('🆘 Помощь')
    markup.add(btn1, btn2, btn3, btn4)
    return markup

# Создаем клавиатуру для возврата в меню
def create_back_menu():
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn = types.KeyboardButton('⬅️ В главное меню')
    markup.add(btn)
    return markup

# Обработчик команды /start и /menu
@bot.message_handler(commands=['start', 'menu'])
def send_welcome(message):
    welcome_text = """
    👋 Привет! Я бот для анализа комментариев о работе управляющей компании.
    
    Выберите действие из меню:
    - 🔍 Анализ комментария: пришлите мне текст, и я его проанализирую
    - ℹ️ О боте: узнайте больше о моих возможностях
    - 📊 Примеры комментариев: посмотрите примеры для анализа
    - � Помощь: как пользоваться ботом
    
    Или просто отправьте мне комментарий для анализа.
    """
    bot.send_message(message.chat.id, welcome_text, reply_markup=create_main_menu())

# Обработчик кнопки "О боте"
@bot.message_handler(func=lambda message: message.text == 'ℹ️ О боте')
def about_bot(message):
    about_text = """
    🤖 <b>О боте</b>
    
    Я использую машинное обучение для анализа комментариев о работе управляющей компании. 
    Я могу определить к каким категориям относится ваш отзыв:
    
    • Нравится скорость отработки заявок
    • Нравится качество выполнения заявки
    • Нравится качество работы сотрудников
    • Понравилось выполнение заявки
    • Вопрос решен
    
    Точность анализа: ~85%
    Версия: 1.0
    """
    bot.send_message(message.chat.id, about_text, parse_mode='HTML', reply_markup=create_back_menu())

# Обработчик кнопки "Примеры комментариев"
@bot.message_handler(func=lambda message: message.text == '📊 Примеры комментариев')
def examples(message):
    examples_text = """
    <b>Примеры комментариев для анализа:</b>
    
    Положительные:
    - "Мастер пришел быстро и все качественно починил"
    - "Спасибо за оперативное решение проблемы"
    - "Работой сотрудников доволен, все сделали хорошо"
    
    Нейтральные/отрицательные:
    - "Ждал мастера 3 дня, но работу выполнили нормально"
    - "Проблему решили, но пришлось несколько раз звонить"
    - "Качество работы нормальное, но сроки подкачали"
    
    Попробуйте отправить мне похожий комментарий!
    """
    bot.send_message(message.chat.id, examples_text, parse_mode='HTML', reply_markup=create_back_menu())

# Обработчик кнопки "Помощь"
@bot.message_handler(func=lambda message: message.text == '🆘 Помощь')
def help(message):
    help_text = """
    <b>📌 Как пользоваться ботом:</b>
    
    1. Отправьте мне текст комментария о работе управляющей компании
    2. Я проанализирую его и покажу, к каким категориям он относится
    3. Для каждого совпадения будет указана вероятность
    
    <b>Советы:</b>
    - Чем подробнее комментарий, тем точнее анализ
    - Бот лучше работает с полными предложениями
    - Используйте кнопку "Примеры", чтобы увидеть шаблоны
    
    Если возникли проблемы - напишите @ваш_username
    """
    bot.send_message(message.chat.id, help_text, parse_mode='HTML', reply_markup=create_back_menu())

# Обработчик кнопки "Анализ комментария"
@bot.message_handler(func=lambda message: message.text == '🔍 Анализ комментария')
def ask_for_comment(message):
    msg = bot.send_message(message.chat.id, "📝 Отправьте мне комментарий для анализа:", reply_markup=create_back_menu())
    bot.register_next_step_handler(msg, classify_comment)

# Обработчик возврата в меню
@bot.message_handler(func=lambda message: message.text == '⬅️ В главное меню')
def back_to_menu(message):
    send_welcome(message)

# Обработчик текстовых сообщений (анализ комментария)
@bot.message_handler(func=lambda message: True, content_types=['text'])
def classify_comment(message):
    try:
        # Проверяем, не является ли сообщение командой
        if message.text.startswith('/'):
            bot.send_message(message.chat.id, "Используйте кнопки меню для навигации", reply_markup=create_main_menu())
            return
            
        comment = message.text
        
        # Получаем предсказания модели
        prediction = model.predict([comment])[0]
        probabilities = model.predict_proba([comment])
        
        # Формируем ответ
        response = f"📄 <b>Комментарий:</b> \"{comment}\"\n\n"
        response += "🔍 <b>Результаты анализа:</b>\n"
        
        # Собираем все вероятности в список
        all_probs = [probabilities[i][0][1] for i in range(len(class_columns))]
        
        has_positive = False
        for i, col in enumerate(class_columns):
            prob = probabilities[i][0][1]
            if prob > 0.5:  # Показываем только категории с вероятностью > 50%
                response += f"✅ {col}: <b>{prob:.1%}</b>\n"
                has_positive = True
        
        if not has_positive:
            response += "❌ Не удалось определить явные положительные категории (все вероятности ≤ 50%)"
        
        # Добавляем общую оценку
        avg_prob = sum(all_probs)/len(all_probs)
        if avg_prob > 0.7:
            sentiment = "👍 Положительный"
        elif avg_prob > 0.4:
            sentiment = "🤔 Нейтральный"
        else:
            sentiment = "👎 Отрицательный"
            
        response += f"\n<b>Общая оценка:</b> {sentiment} ({avg_prob:.1%})"
        
        bot.send_message(message.chat.id, response, parse_mode='HTML', reply_markup=create_main_menu())
        
    except Exception as e:
        error_text = f"⚠️ <b>Ошибка</b>\n\nНе удалось проанализировать комментарий: {str(e)}\n\nПопробуйте изменить формулировку или использовать примеры из меню."
        bot.send_message(message.chat.id, error_text, parse_mode='HTML', reply_markup=create_main_menu())

print("Бот запущен и готов к работе!")
bot.polling()

Модель успешно загружена
Бот запущен и готов к работе!
